# Bitcoin price visualiser

In [1]:
import os
import quandl
import pickle
import plotly
import numpy as pd
import pandas as pd

import config

import matplotlib.pyplot as plt

In [2]:
quandl.ApiConfig.api_key = config.key

## Cache bitcoin data

In [5]:
def get_quandl_data(quandl_code):
    """
    Provide price data for given quandl_code via caching mechanism 
    """
    cache_path = "{}.pkl".format(quandl_code).replace("/", "-")
    
    # load price data if it exists
    try:
        file = open(cache_path, "rb")
        df = pickle.load(file)
        print(f"Loaded {quandl_code} data from cache")
    # othewise cache the data
    except (OSError, IOError):
        df = quandl.get(quandl_code, returns="pandas")
        df.to_pickle(cache_path)
        print(f"Cached {quandl_code} data to {cache_path}")
    return df

In [8]:
btc_usd_kraken = get_quandl_data("BCHARTS/KRAKENUSD")

Loaded BCHARTS/KRAKENUSD data from cache


In [10]:
btc_usd_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709
